In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
daily_totals_path = "../../../results/belgium/airport/daily_totals.parquet"
departure_hours_path = "../../../results/belgium/airport/departure_hours.parquet"
group_sizes_path = "../../../results/belgium/airport/group_sizes.parquet"
passenger_profiles_path = "../../../results/belgium/airport/passenger_profiles.parquet"
survey_path = "../../../results/belgium/airport/survey.parquet"
spatial_path = "../../../results/belgium/census/spatial.parquet"
sectors_path = "../../../results/belgium/census/sectors.parquet"
municipalities_path = "../../../results/belgium/census/municipalities.parquet"

output_path = "/home/shoerl/temp/maia"

### Daily passengers

In [ ]:
df = pd.read_parquet(daily_totals_path)

figure = px.line(df, x = "date", y = "passengers")

with open("{}/daily_totals.png".format(output_path), "wb") as f:
    f.write(figure.to_image("png", width = 600, height = 400, scale = 4))

print("Mean:", df["passengers"].mean())
figure.show()

### Departure hours

In [ ]:
df = pd.read_parquet(departure_hours_path)
df = df.set_index(["passenger_profile", "departure_hour"])
df = df.reindex(pd.MultiIndex.from_product([[
    "business", "economy"
], list(range(24))], names = ["passenger_profile", "departure_hour"])).reset_index().fillna(0.0)

figure = px.line(df, x = "departure_hour", y = "weight", color = "passenger_profile")

with open("{}/departure_hours.png".format(output_path), "wb") as f:
    f.write(figure.to_image("png", width = 600, height = 400, scale = 4))

figure.show()

### Group sizes

In [ ]:
df = pd.read_parquet(group_sizes_path)

figure = px.bar(df, x = "group_size", y = "weight")

with open("{}/group_sizes.png".format(output_path), "wb") as f:
    f.write(figure.to_image("png", width = 600, height = 400, scale = 4))

figure.show()

In [ ]:
(df["group_size"] * df["weight"]).sum() / df["weight"].sum()

### Passenger profiles

In [ ]:
df = pd.read_parquet(passenger_profiles_path)
df

### Survey

In [ ]:
df = pd.read_parquet(survey_path)
print("Observations:", len(df))
df.head(5)

In [ ]:
print("Respondents:", df["annual_respondents"].sum())

In [ ]:
df_plot = df.groupby(["sex", "age_class"])["annual_respondents"].sum().reset_index()
df_plot["sex"] = df_plot["sex"].replace({ 1: "male", 2: "female" })

figure = px.bar(df_plot, x = "age_class", y = "annual_respondents", color = "sex", barmode = "group")

with open("{}/survey.png".format(output_path), "wb") as f:
    f.write(figure.to_image("png", width = 600, height = 400, scale = 4))

figure.show()

### Spatial data

In [ ]:
df_municipalities = gpd.read_parquet(spatial_path)
df_municipalities = df_municipalities.dissolve("municipality_id").reset_index()

In [ ]:
df_plot["annual_respondents"].sum()

In [ ]:
df_plot = pd.merge(
    df_municipalities, 
    df.groupby("municipality_id")["annual_respondents"].sum().reset_index(),
    on = "municipality_id")

df_plot = df_plot[df_plot["annual_respondents"] > 0]
df_plot["density"] = df_plot["annual_respondents"] / (df_plot["geometry"].area * 1e-6)

plt.figure(dpi = 300)
ax = df_plot.plot("annual_respondents", legend = True, vmax = 1000, legend_kwds = { "label": "Annual respondents", "orientation": "vertical" }, ax = plt.gca())
ax.set_axis_off()

plt.savefig("{}/survey_map.png".format(output_path))

### Spatial

In [ ]:
df_spatial = gpd.read_parquet(spatial_path)
print("Municipalities:", len(df_spatial["municipality_id"].unique()))
print("Sectors:", len(df_spatial["sector_id"].unique()))

### Sectors

In [ ]:
df_sectors = pd.read_parquet(sectors_path)
df_sectors = pd.merge(df_spatial, df_sectors[["sector_id", "weight"]], on = "sector_id")

In [ ]:
plt.figure(dpi = 300)
ax = df_sectors.plot("weight", legend = True, vmax = 6000, legend_kwds = { "label": "Inhabitants by sector", "orientation": "vertical" }, ax = plt.gca())
ax.set_axis_off()

plt.savefig("{}/sectors_map.png".format(output_path))

### Municipalities

In [ ]:
df_municipality_data = pd.read_parquet(municipalities_path)

In [ ]:
df_plot = df_municipality_data.groupby(["age_class", "sex"])["weight"].sum().reset_index()
df_plot["sex"] = df_plot["sex"].replace({ 1: "male", 2: "female" })
df_plot["age_class"] = df_plot["age_class"].astype(str)

figure = px.bar(df_plot, x = "age_class", y = "weight", color = "sex", barmode = "group")

with open("{}/municipalities_sex_age.png".format(output_path), "wb") as f:
    f.write(figure.to_image("png", width = 600, height = 400, scale = 4))

figure.show()

In [ ]:
df_map = pd.merge(df_municipalities[["municipality_id", "geometry"]], 
    df_municipality_data.groupby("municipality_id")["weight"].sum().reset_index())

df_map["weightk"] = df_map["weight"] * 1e-3
df_map["density"] = df_map["weight"] / (df_map["geometry"].area / 1e6)

plt.figure(dpi = 300)

plt.subplot(1, 2, 1)
ax = df_map.plot("weightk", legend = True, vmax = 100, legend_kwds = { "label": "Inhabitants [x1000]", "orientation": "horizontal" }, ax = plt.gca())
ax.set_axis_off()

plt.subplot(1, 2, 2)
ax = df_map.plot("density", legend = True, vmax = 2000, legend_kwds = { "label": "Density [pax/km2]", "orientation": "horizontal" }, ax = plt.gca())
ax.set_axis_off()

plt.savefig("{}/municipalities_map.png".format(output_path))

In [ ]:
df_total = df_municipality_data.groupby("municipality_id")["weight"].sum().reset_index(name = "total")

df_age = df_municipality_data[["municipality_id", "age", "weight"]].copy()
df_age["age"] = df_age["age"] * df_age["weight"]
df_age = df_age.groupby("municipality_id")["age"].sum().reset_index()
df_age = pd.merge(df_age, df_total, on = "municipality_id")
df_age["age"] = df_age["age"] / df_age["total"]
df_age = pd.merge(df_municipalities, df_age)

plt.figure(dpi = 300)
ax = df_age.plot("age", legend = True, legend_kwds = { "label": "Mean age", "orientation": "vertical" }, ax = plt.gca())
ax.set_axis_off()

plt.savefig("{}/municipalities_mean_age.png".format(output_path))